In [1]:
import numpy as np
import sympy
from sympy import exp,besselk,pi,Function,sqrt,Heaviside
from mpmath import apery

In [19]:
def fdiff(T):

    mass = 500.
    dof = -2
    Zeta3 = apery
    x = T/mass
    if x <= 0.1:
        dneq = mass**2*exp(-1/x)*(256 + 3*x*(288+5*x*(94+49*x)))/(512*sqrt(2)*pi**(3/2)*sqrt(x))
    elif x <= 1.5:
        dneq = mass**2*(besselk(1,1/x)+3*x*besselk(2,1/x))/(2*pi**2*x)
    else:
        if dof > 0: dneq = 3*Zeta3*T**2/pi**2   #Relativistic Bosons
        if dof < 0: dneq = 3*(3./4.)*Zeta3*T**2/pi**2   #Relativistic Fermions

    return dneq*abs(dof)

class nEQc(sympy.Function):
    """Returns the equilibrium number density at temperature T. Returns zero for non-thermal components"""

    nargs = 3
            
    @classmethod
    def eval(cls,T,mass,dof):
        
        Zeta3 = apery
        x = T/mass
    
        neq = Heaviside(0.1-x,1.)*(mass**3*(x/(2*pi))**(3./2.)*exp(-1/x)*(1. + (15./8.)*x + (105./128.)*x**2))
        neq += Heaviside(1.5-x,1.)*Heaviside(x-0.1,0.)*(mass**3*x*besselk(2,1/x)/(2*pi**2))
        neq += Heaviside(x-1.5,0.)*(Heaviside(dof,1.)*(Zeta3*T**3/pi**2)+Heaviside(-dof,0.)*(3./4.)*(Zeta3*T**3/pi**2))
        neq = neq*abs(dof)
        
        return neq
    
    def _eval_is_real(self):
        return self.args[0].is_real
    
T,mass,dof = sympy.symbols('T,mass,dof')
nEQ = sympy.lambdify([T,mass,dof],nEQc(T,mass,dof),modules=[{'DiracDelta' : lambda x: 0.},'numpy','sympy'])
dnEQ = sympy.lambdify([T,mass,dof],nEQc(T,mass,dof).diff(T),modules=[{'DiracDelta' : lambda x: 0.},'numpy','sympy'])

In [18]:
nEQc(T,mass,dof)

(T*mass**2*Heaviside(-T/mass + 1.5, 1.0)*Heaviside(T/mass - 0.1, 0.0)*besselk(2, mass/T)/(2*pi**2) + 0.353553390593274*pi**(-1.5)*mass**3*(T/mass)**1.5*(0.8203125*T**2/mass**2 + 1.875*T/mass + 1.0)*exp(-mass/T)*Heaviside(-T/mass + 0.1, 1.0) + (0.901542677369696*T**3*Heaviside(-dof, 0.0)/pi**2 + 1.20205690315959*T**3*Heaviside(dof, 1.0)/pi**2)*Heaviside(T/mass - 1.5, 0.0))*Abs(dof)

In [21]:
dnEQ(50.,500.,-2.).evalf(),dnEQ(750.,500.,-2.).evalf()

(6.36018018758805, 328653.271502128)

In [6]:
fdiff(50.).evalf(),fdiff(750.).evalf()

(6.36018018758805, 328653.271502128)

In [7]:
nEQc.diff(T)

1

In [8]:
nEQc(T).diff(T)

3952847.07521047*pi**(-1.5)*T**(-0.5)*(3.28125e-6*T**2 + 0.00375*T + 1.0)*exp(-500.0/T)*Heaviside(0.1 - 0.002*T, 1.0) + 11858.5412256314*pi**(-1.5)*T**0.5*(3.28125e-6*T**2 + 0.00375*T + 1.0)*exp(-500.0/T)*Heaviside(0.1 - 0.002*T, 1.0) + 0.00360617070947878*T**3*DiracDelta(0.002*T - 1.5)/pi**2 + 5.40925606421817*T**2*Heaviside(0.002*T - 1.5, 0.0)/pi**2 - 500.0*T*DiracDelta(1.5 - 0.002*T)*Heaviside(0.002*T - 0.1, 0.0)*besselk(2, 500.0/T)/pi**2 + 500.0*T*DiracDelta(0.002*T - 0.1)*Heaviside(1.5 - 0.002*T, 1.0)*besselk(2, 500.0/T)/pi**2 + 7905.69415042095*pi**(-1.5)*T**1.5*(6.5625e-6*T + 0.00375)*exp(-500.0/T)*Heaviside(0.1 - 0.002*T, 1.0) - 15.8113883008419*pi**(-1.5)*T**1.5*(3.28125e-6*T**2 + 0.00375*T + 1.0)*exp(-500.0/T)*DiracDelta(0.1 - 0.002*T) + 250000.0*Heaviside(1.5 - 0.002*T, 1.0)*Heaviside(0.002*T - 0.1, 0.0)*besselk(2, 500.0/T)/pi**2 - 125000000.0*(-besselk(1, 500.0/T)/2 - besselk(3, 500.0/T)/2)*Heaviside(1.5 - 0.002*T, 1.0)*Heaviside(0.002*T - 0.1, 0.0)/(pi**2*T)